In [5]:
import torch
import os
import scanpy as sp
import numpy as np
import pandas as pd 

In [2]:
import utils
import argparse


parser = argparse.ArgumentParser(description='Main program for sencells')

parser.add_argument('--output_dir', type=str, default='./outputs', help='')
parser.add_argument('--exp_name', type=str, default='data2', help='')
parser.add_argument('--device_index', type=int, default=0, help='')
parser.add_argument('--retrain', action='store_true', default=False, help='')
parser.add_argument('--timestamp', type=str,  default="", help='use default')

parser.add_argument('--seed', type=int, default=40, help='different seed for different experiments')
parser.add_argument('--n_genes', type=str, default='full', help='set 3000, 8000 or full')
parser.add_argument('--ccc', type=str, default='type2', help='type1: cell-cell edge with weight in 0 and 1. type2: cell-cell edge with weight in 0 to 1. type3: no cell-cell edge')
parser.add_argument('--gene_set', type=str, default='full', help='senmayo or fridman or cellage or goterm or goterm+fridman or senmayo+cellage or senmayo+fridman or senmayo+fridman+cellage or full')

parser.add_argument('--gat_epoch', type=int, default=30, help='use default')


# --------------------------------------------------------------------------------------------------- #
# Write these code to fit our data input. This is for our @Yi, @Ahmed, and @Hu.
parser.add_argument('--input_data_count', type=str, default="/bmbl_data/huchen/deepSAS_data/fixed_data_0525.h5ad", help='it is a path to a adata object (.h5ad)')
parser.add_argument('--input_data_CCC_file', type=str, default="", help='it is a path to a CCC file (.csv or .npy)')
# --------------------------------------------------------------------------------------------------- #


# --------------------------------------------------------------------------------------------------- #
# Subsampling argument for our following version. Please check this @Yi, @Ahmed, and @Hu.
parser.add_argument('--subsampling', action='store_true', default=False, help='subsampling')
# --------------------------------------------------------------------------------------------------- #


# For @Hao
# Hao: Just delete these 3 parameters.
# --------------------------------------------------------------------------------------------------- #
# is this sencell_num parameter not used? Please check this @Hao.
parser.add_argument('--sencell_num', type=int, default=600, help='use default')
# is this sengene_num parameter not used? Please check this @Hao.
parser.add_argument('--sengene_num', type=int, default=200, help='use default')
# is this sencell_epoch parameter not used? Please check this @Hao.
parser.add_argument('--sencell_epoch', type=int, default=40, help='use default')
# --------------------------------------------------------------------------------------------------- #




parser.add_argument('--cell_optim_epoch', type=int, default=50, help='use default')


# For @Hao
# Hao: This is the emb size for GAT hidden embedding size
# --------------------------------------------------------------------------------------------------- #
# is this emb_size parameter for what, for GAT? is default 12? Please check this @Hao.
parser.add_argument('--emb_size', type=int, default=12, help='use default')
# --------------------------------------------------------------------------------------------------- #



parser.add_argument('--batch_id', type=int, default=0, help='use default')

args = parser.parse_args([])

adata, cluster_cell_ls, cell_cluster_arr, celltype_names = utils.load_data_newfix("/bmbl_data/huchen/deepSAS_data/fixed_data_0525.h5ad")

new_data, markers_index,\
    sen_gene_ls, nonsen_gene_ls, gene_names = utils.process_data(
        adata, cluster_cell_ls, cell_cluster_arr,args)
print(new_data)

load_data_newfix ...
Number of cells: 24125
Number of genes: 15844
The number of cell types 32
celltype names: ['Fibrotic fibroblast', 'AT2', 'EC Venous', 'EC Arterial', 'Dendritic cells', 'CD4+ T Cells', 'KRT5-/KRT17+ cells', 'EC General capillary', 'Monocyte-derived macrophage', 'CD8+ T Cells', 'Smooth muscle cells', 'Plasma cells', 'Ciliated', 'SPP1+ macrophages', 'MUC5B+ club', 'Alveolar fibroblasts', 'Alveolar macrophages', 'SCGB3A2+/SCGB1A1+ club', 'Monocyte', 'Mast cells', 'NK cells', 'Pericyte', 'Goblet', 'AT1', 'EC Lymphatic', 'B cells', 'Inflammatory fibroblasts', 'AT2 transitional', 'Peribronchial fibroblasts', 'Adventitial fibroblasts', 'Proliferating fibroblasts', 'Basal']
---------------------------  ----
Fibrotic fibroblast          2433
AT2                          2195
EC Venous                    2172
EC Arterial                  1847
Dendritic cells              1425
CD4+ T Cells                 1211
KRT5-/KRT17+ cells           1145
EC General capillary          950

In [3]:
pd.DataFrame(adata.obs['clusters'].value_counts()).to_csv("cell_types.csv")

In [4]:
file_path = "/bmbl_data/huchen/sencell_data1/outputs/data1/data1_sencellgene-epoch4.data"

# Load the saved object
loaded_data = torch.load(file_path)

# Unpack the loaded data
sencell_dict, sen_gene_ls, attention_scores, edge_index_selfloop = loaded_data

print(f"Number of SnCs: {len(sencell_dict.keys())}")
print(f"Number of SnGs: {len(sen_gene_ls)}")

Number of SnCs: 777
Number of SnGs: 299


In [7]:
sencell_indexs=list(sencell_dict.keys())
sencell_cluster = []
for i in sencell_indexs:
    ct=new_data.obs.iloc[i-new_data.shape[1]].clusters
    sencell_cluster.append(ct)


In [11]:
sencell_df = pd.DataFrame({'sencell_index': sencell_indexs, 'sencell_cluster': sencell_cluster})
sencell_df.head()

,sencell_index,sencell_cluster
0,15851,SPP1+ macrophages
1,15853,SPP1+ macrophages
2,15870,AT2
3,15898,SPP1+ macrophages
4,15900,AT2


In [13]:
pd.DataFrame(new_data.obs['clusters'].value_counts())

,clusters
Fibrotic fibroblast,2433
AT2,2195
EC Venous,2172
EC Arterial,1847
Dendritic cells,1425
CD4+ T Cells,1211
KRT5-/KRT17+ cells,1145
EC General capillary,950
Monocyte-derived macrophage,926
CD8+ T Cells,892


In [14]:
pd.DataFrame(sencell_df['sencell_cluster'].value_counts())

,sencell_cluster
Fibrotic fibroblast,123
AT2,117
CD4+ T Cells,73
KRT5-/KRT17+ cells,59
EC Venous,50
Ciliated,46
Monocyte-derived macrophage,41
EC Arterial,32
Dendritic cells,30
Alveolar macrophages,30


In [5]:
Yi_file_path = "/bmbl_data/huchen/sencell_data1/outputs/data1/Data_23024_sencellgene-epoch4.data"   

# Load the saved object
Yi_loaded_data = torch.load(Yi_file_path)

# Unpack the loaded data
Yi_sencell_dict, Yi_sen_gene_ls, Yi_attention_scores, Yi_edge_index_selfloop = Yi_loaded_data

print(f"Number of SnCs: {len(Yi_sencell_dict.keys())}")
print(f"Number of SnGs: {len(Yi_sen_gene_ls)}")


Number of SnCs: 664
Number of SnGs: 281


In [6]:
selected_ct=['Fibrotic fibroblast',
    'EC Venous',
    'CD4+ T Cells',
    'Plasma cells',
    'AT2',
    'Smooth muscle cells',
    'EC General capillary',
    'Dendritic cells']

In [7]:
sen_gene_ls

tensor([15738, 15733, 15829, 15776, 15794, 15772, 15770, 15727, 15836, 15630,
        15721, 15724, 15722, 15745, 15731, 15819, 15801, 15632, 15755, 15703,
        15652, 15824, 15825, 15830, 15756, 15807, 15611, 15695, 15635, 15560,
        15725, 15768, 15571, 15634, 15558, 15545, 15561, 15708, 15806, 15586,
        15811, 15577, 15773, 15741, 15736, 15596, 15600, 15602, 15690, 15781,
        15597, 15580, 15609, 15633, 15815, 15711, 15735, 15792, 15805, 15624,
        15598, 15588, 15729, 15693, 15716, 15752, 15715, 15655, 15714, 15786,
        15618, 15804, 15751, 15706, 15653, 15814, 15812, 15710, 15764, 15769,
        15709, 15604, 15548, 15739, 15835, 15707, 15663, 15763, 15649, 15657,
        15810, 15659, 15757, 15799, 15628, 15616, 15728, 15808, 15782, 15667,
        15778, 15648, 15700, 15620, 15656, 15831, 15639, 15647, 15559, 15798,
        15818, 15843, 15546, 15702, 15701, 15625, 15771, 15783, 15692, 15839,
        15754, 15593, 15565, 15670, 15640, 15617, 15726, 15760, 

Intersection between original senescent marker genes and output senescent marker genes

In [8]:
sen_gene_ls

filtered_gene_names = []
for gene_idx in sen_gene_ls:
    gene_idx = gene_idx.item()
    filtered_gene_names.append(new_data.var.index[gene_idx])
gene_idx2names = {'gene_idx': sen_gene_ls, 'gene_name': filtered_gene_names}

gene_idx2names_df = pd.DataFrame(gene_idx2names)
gene_idx2names_df.to_csv("predict_sene_makrers.csv", index=False)
inital_masker = pd.read_csv("initial_sene_markers.csv")
# Not in hallmarks, generate from model output
predict_sens_list = list(set(gene_idx2names_df['gene_name']).difference(set(inital_masker['gene_name'])))

print(len(predict_sens_list))




Yi_filtered_gene_names = []
for gene_idx in Yi_sen_gene_ls:
    gene_idx = gene_idx.item()
    Yi_filtered_gene_names.append(new_data.var.index[gene_idx])
Yi_gene_idx2names = {'gene_idx': Yi_sen_gene_ls, 'gene_name': Yi_filtered_gene_names}

Yi_gene_idx2names_df = pd.DataFrame(Yi_gene_idx2names)
Yi_gene_idx2names_df.to_csv("Yi_predict_sene_makrers.csv", index=False)

Yi_predict_sens_list = list(set(Yi_gene_idx2names_df['gene_name']).difference(set(inital_masker['gene_name'])))

print(len(Yi_predict_sens_list))


28
18


In [9]:
len(set(Yi_predict_sens_list) & set(predict_sens_list))

6

In [11]:
def identify_sengene_v2(new_data,
                        sencell_dict):
    # prepare adata for deg
    adata_deg=new_data.copy()
    sp.pp.normalize_total(adata_deg, target_sum=1e4)
    sp.pp.log1p(adata_deg)
    sp.pp.scale(adata_deg)

    # selected_ct=['Fibrotic fibroblast',
    #   'EC Venous',
    #   'CD4+ T Cells',
    #   'Plasma cells',
    #   'AT2',
    #   'Smooth muscle cells',
    #   'EC General capillary',
    #   'Dendritic cells']
    # No Smooth muscle cells detected in sencell_dict


    sencell_indexs=list(sencell_dict.keys())
    
    # Cell index where ct in select_ct
    sencell_indexs_updated=[]
    for i in sencell_indexs:
        ct=adata_deg.obs.iloc[i-adata_deg.shape[1]].clusters
        if ct in selected_ct:
            sencell_indexs_updated.append(i)
            
    sencell_indexs=sencell_indexs_updated
    
    # create SnC column
    # NOTE: is_sen: normal / SnC
    row_indices= np.array(sencell_indexs)-new_data.shape[1]
    new_column = np.array(['normal']*new_data.shape[0])
    new_column[row_indices] = 'SnC'
    adata_deg.obs['is_sen'] = new_column
    adata_deg.obs['is_sen']=adata_deg.obs['is_sen'].astype(str).astype('category')
    # Verify the result
    # print(adata_deg.obs['is_sen'])
    
    cell_types = adata_deg.obs['clusters'].unique()
    
    all_genes_names=list(new_data.var.index)
    results_genes=[]
    deg_results={}
    deg_results_full={}
    
    for cell_type in cell_types:
        adata_deg_sub=adata_deg[adata_deg.obs['clusters']==cell_type]
        if 'SnC' in adata_deg_sub.obs['is_sen'].values:
            value_counts = adata_deg_sub.obs['is_sen'].value_counts()
            # 只考虑老化细胞数大于5的cell type
            if value_counts['SnC']<=5:
                continue
            print(f"{cell_type}, Number of SnCs: {value_counts['SnC']}")    
            sp.tl.rank_genes_groups(adata_deg_sub, groupby='is_sen', groups=['SnC'], 
                                    reference='normal', method='wilcoxon')

            # Extract the results into a DataFrame
            degs = pd.DataFrame({
                'gene': adata_deg_sub.uns['rank_genes_groups']['names']['SnC'],
                'p_val': adata_deg_sub.uns['rank_genes_groups']['pvals']['SnC'],
                'logFC': adata_deg_sub.uns['rank_genes_groups']['logfoldchanges']['SnC'],
                'p_val_adj': adata_deg_sub.uns['rank_genes_groups']['pvals_adj']['SnC']
            })
            
            degs=degs.sort_values(by='logFC')
            degs['prediction'] = degs['gene'].isin(predict_sens_list).astype(int)
            subdegs = degs[degs['prediction'] == 1]
            save_ct_name = cell_type.replace(' ', '_')
            degs.to_csv(f"outputs/DEG_res/{save_ct_name}_DEG_results.csv", index=False)
            subdegs.to_csv(f"outputs/DEG_res/{save_ct_name}_sub_DEG_results.csv", index=False)
            deg_results_full[cell_type]=degs
            num=len(degs)
            print(f'Get {num} degs!')
    return adata_deg, sencell_indexs, deg_results_full


adata_deg, sencell_index_in_deg, deg_results_full=identify_sengene_v2(new_data,
                        sencell_dict)

AT2, Number of SnCs: 117


/bmbl_data/huchen/sencell_data1/.venv/lib/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:580: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/bmbl_data/huchen/sencell_data1/.venv/lib/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:420: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(
/bmbl_data/huchen/sencell_data1/.venv/lib/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:580: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}


Get 15844 degs!
Plasma cells, Number of SnCs: 20


/bmbl_data/huchen/sencell_data1/.venv/lib/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:420: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(
/bmbl_data/huchen/sencell_data1/.venv/lib/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:580: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}


Get 15844 degs!
CD4+ T Cells, Number of SnCs: 73


...

/bmbl_data/huchen/sencell_data1/.venv/lib/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:420: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(
/bmbl_data/huchen/sencell_data1/.venv/lib/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:580: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}


Get 15844 degs!
EC Venous, Number of SnCs: 50


/bmbl_data/huchen/sencell_data1/.venv/lib/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:420: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(
/bmbl_data/huchen/sencell_data1/.venv/lib/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:580: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}


Get 15844 degs!
EC General capillary, Number of SnCs: 22
Get 15844 degs!


/bmbl_data/huchen/sencell_data1/.venv/lib/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:420: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(


In [12]:
# new_data=sp.read_h5ad()#####your h5ad file
#adata, cluster_cell_ls, cell_cluster_arr, celltype_names = utils.load_data_newfix(args.input_data_count)
gene_cell = new_data.X.toarray().T

    ####build dict for each sncs for each cell type
sencell_indexs=list(sencell_dict.keys())
ct_sencell_indexs={}
row_numbers=np.array(sencell_indexs)-new_data.shape[1]

for i in row_numbers:
    ct_=new_data.obs.iloc[i]['clusters']
    if ct_ in selected_ct:
        if ct_ in ct_sencell_indexs:
            ct_sencell_indexs[ct_].append(i+new_data.shape[1])
        else:
            ct_sencell_indexs[ct_]=[i+new_data.shape[1]]

In [13]:
len(ct_sencell_indexs.keys())

8

In [14]:
import scanpy as sc

def getHVGindex(adata):
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
    return adata

HVG_adata = getHVGindex(new_data.copy())
print(HVG_adata)

/bmbl_data/huchen/sencell_data1/.venv/lib/python3.9/site-packages/pandas/core/util/hashing.py:342: RuntimeWarning: invalid value encountered in cast
  vals.astype(str).astype(object), hash_key, encoding
/bmbl_data/huchen/sencell_data1/.venv/lib/python3.9/site-packages/pandas/core/util/hashing.py:342: RuntimeWarning: invalid value encountered in cast
  vals.astype(str).astype(object), hash_key, encoding
/bmbl_data/huchen/sencell_data1/.venv/lib/python3.9/site-packages/pandas/core/util/hashing.py:342: RuntimeWarning: invalid value encountered in cast
  vals.astype(str).astype(object), hash_key, encoding


AnnData object with n_obs × n_vars = 24125 × 15844
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'Sample', 'Treatment', 'Injury', 'Status', 'Area', 'Type', 'Age', 'Age_Status', 'Sex', 'Treatment_Age_Status', 'Treatment_Age_Status_Sex', 'Treatment_Age_Status_Sex_Area', 'Names_Sample', 'Names_Age_Status', 'Names_Sample_Age_Status', 'nCount_SCT', 'nFeature_SCT', 'integrated_snn_res.0.2', 'seurat_clusters', 'integrated_snn_res.0.5', 'predicted.id', 'prediction.score.SCGB3A2..SCGB1A1..club', 'prediction.score.Plasma.cells', 'prediction.score.CD4..T.Cells', 'prediction.score.Goblet', 'prediction.score.B.cells', 'prediction.score.EC.Arterial', 'prediction.score.Smooth.muscle.cells', 'prediction.score.Ciliated', 'prediction.score.Proliferating.fibroblasts', 'prediction.score.AT2', 'prediction.score.SPP1..macrophages', 'prediction.score.Monocyte.derived.macrophage', 'prediction.score.Alveolar.fibroblasts', 'prediction.score.Mast.cells', 'prediction.score.Inflammatory.fibrob

In [15]:
HVG_var = HVG_adata.var.reset_index()
high_gene_indexs = list(HVG_var[HVG_var['highly_variable'] == True].index)
print(high_gene_indexs)

[23, 25, 27, 30, 32, 33, 34, 37, 58, 82, 87, 88, 99, 110, 115, 135, 147, 150, 152, 153, 154, 174, 186, 194, 195, 197, 199, 204, 221, 231, 234, 237, 238, 240, 242, 244, 245, 246, 250, 251, 253, 254, 255, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 269, 277, 280, 281, 282, 287, 288, 290, 294, 301, 309, 311, 312, 313, 316, 317, 318, 321, 325, 334, 341, 349, 352, 354, 355, 356, 362, 363, 367, 368, 371, 379, 385, 389, 407, 410, 411, 427, 429, 430, 435, 439, 445, 453, 469, 473, 489, 494, 500, 526, 537, 539, 542, 543, 558, 568, 587, 594, 595, 598, 606, 612, 632, 652, 657, 670, 672, 674, 678, 679, 684, 686, 700, 701, 736, 738, 743, 752, 753, 757, 760, 762, 764, 765, 767, 768, 769, 771, 772, 775, 779, 792, 793, 798, 804, 805, 822, 830, 837, 840, 843, 854, 858, 861, 863, 864, 867, 869, 873, 874, 882, 895, 906, 914, 931, 932, 968, 978, 980, 993, 1017, 1023, 1029, 1032, 1035, 1052, 1054, 1085, 1093, 1096, 1097, 1100, 1103, 1109, 1137, 1148, 1162, 1206, 1210, 1212, 1218, 1231, 1245, 1255, 127

In [16]:
def identify_sengene_v1(sencell_dict,gene_cell,edge_index_selfloop,attention_scores,
                        sen_gene_ls,ct_sencell_indexs, high_gene_indexs):
    attention_scores=attention_scores.detach().to('cpu')
    edge_index_selfloop=edge_index_selfloop.detach().to('cpu')

 
    num_genes = gene_cell.shape[0]
    num_cells = gene_cell.shape[1]
    cell_mask_template = torch.zeros(num_genes + num_cells, dtype=torch.bool)

    # NOTE: gene_scores for each gene, actually only for sen_gen_ls is enough
    # return gene_scores
    gene_scores=[]
    for gene_index in range(num_genes):
        # NOTE: high varible genes top 3k
        if gene_index in sen_gene_ls or gene_index in high_gene_indexs:
            # 保存一个基因在8个ct的snc上的score
            ct_scores=[]  
            for key,value in ct_sencell_indexs.items():
            
                cell_index=value
                cell_index=torch.tensor(cell_index)
                
                cell_mask = cell_mask_template.clone()
                cell_mask[cell_index] = True
                
                connected_cells=edge_index_selfloop[0][edge_index_selfloop[1] == gene_index]
                
                if len(connected_cells[cell_mask[connected_cells]])==0:
                    ct_scores.append(torch.tensor(0))
                    # print(f'no sencell in this gene {gene_index}')
                else:
                    # print('caculate attention')
                #     tmp=attention_scores[edge_index_selfloop[1] == gene_index]
                #     attention_edge=torch.sum(tmp[cell_mask[connected_cells]],axis=1)
                #     cell_mask_normal=~cell_mask
                #     attention_edge_normal=torch.sum(attention_scores[edge_index_selfloop[1] == gene_index][cell_mask_normal[connected_cells]],axis=1)
                # #     #print(cell_mask,cell_mask_normal)
                # #     # print(attention_edge,attention_edge_normal)
                #     attention_s=torch.mean(attention_edge)
                #     attention_normal=torch.mean(attention_edge_normal)
                    
                #     attention_s=attention_s-attention_normal
                # #     # print(attention_s)
                # #     # ct_scores[key] = attention_s.item()
                #     ct_scores.append(attention_s)
                    ct_scores.append(torch.tensor(0.1))
        # 只计算老化基因还是对所有基因
        # if gene_index in sen_gene_ls:
        #     # print("sng:",attention_s)
            gene_scores.append(ct_scores)
        else:
            gene_scores.append([torch.tensor(0)] * 8)

        
    return gene_scores

gene_scores=identify_sengene_v1(
        sencell_dict,new_data.X.T,edge_index_selfloop,attention_scores,sen_gene_ls,ct_sencell_indexs, high_gene_indexs)

In [30]:
gene_scores=torch.tensor(gene_scores)
filtered_gene_scores=[]
filtered_gene_names=[]
for i in range(gene_scores.shape[0]):
    if torch.sum(gene_scores[i])!=0:
        filtered_gene_names.append(new_data.var.index[i])
        filtered_gene_scores.append(gene_scores[i])

filtered_gene_scores=torch.stack(filtered_gene_scores)

In [32]:
filtered_gene_scores.shape

torch.Size([2268, 8])

In [33]:
def calculate_outliers(scores):
    counts=0
    snc_index=[]

    scores_ls=[]
    
    Q1 = np.percentile(scores, 25)
    Q3 = np.percentile(scores, 75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    for i,score in enumerate(scores): 
        if score > upper_bound:
            counts+=1
            snc_index.append(i)
            scores_ls.append(score)
    
    return counts,snc_index,scores_ls

gene_deepsas={}

gene_names=list(new_data.var_names)
gene_names_sub=[gene_names[i] for i in sen_gene_ls]

for i,(key,value) in enumerate(ct_sencell_indexs.items()):
    scores=filtered_gene_scores[:,i]
    print(scores.shape)
    counts,sng_index,scores_ls=calculate_outliers(scores)
    # 计算在每个cluster里面有多少离群gene
    # 就是table1
    print(key,counts)
    sng_names=[[filtered_gene_names[i],float(scores_ls[j])] for j,i in enumerate(sng_index)]
    gene_deepsas[key]=sng_names


rows = []
for cell_type, genes in gene_deepsas.items():
    for gene, score in genes:
        rows.append([cell_type, gene, score])

df1 = pd.DataFrame(rows, columns=['Cell Type', 'Gene Name', 'Score']) ###table 1

df1.to_csv("repro_table1.csv",index=0)

torch.Size([2268])
AT2 23
torch.Size([2268])
CD4+ T Cells 5
torch.Size([2268])
Dendritic cells 5
torch.Size([2268])
Fibrotic fibroblast 24
torch.Size([2268])
Smooth muscle cells 11
torch.Size([2268])
EC Venous 12
torch.Size([2268])
Plasma cells 48
torch.Size([2268])
EC General capillary 12


In [34]:
filtered_degs={}
for key,value in deg_results_full.items():
    filtered_genes=value[(value['logFC']>=0.25)]['gene'].tolist()
    #print(key, filtered_genes)
    filtered_degs[key]=filtered_genes
    print(key,len(filtered_genes))
    
results=[]
for key,value in deg_results_full.items():
    deg_results_full[key]['cell type']=key
    results.append(deg_results_full[key])

pd.concat(results).to_csv("./1.csv")
gene_deepsas_names={}
for key,value in gene_deepsas.items():
    gene_names=[line[0] for line in value]
    gene_deepsas_names[key]=gene_names
gene_deepsas_names

# For table 2
overlap_genes={}
for ct in selected_ct:
    ls1=set(gene_deepsas_names[ct])
    ls2=set(filtered_degs[ct])
    ls3=ls1.intersection(ls2)
    print(ct, len(ls3))
    overlap_genes[ct]=list(ls3)    

table2=[]
for ct in selected_ct:
    t=deg_results_full[ct]
    t=t[t['gene'].isin(overlap_genes[ct])]
    t['cell type']=ct
    table2.append(t)

table2=pd.concat(table2)

table2.to_csv("repro_table2.csv",index=0)
# The genes in table 3 is unique gene only occur in one cell type, I use excel to manually select genes in table3
# No code to generate table3

AT2 6354
Plasma cells 8088
CD4+ T Cells 4371
Smooth muscle cells 7444
Dendritic cells 6784
Fibrotic fibroblast 6126
EC Venous 7324
EC General capillary 7433
Fibrotic fibroblast 9
EC Venous 7
CD4+ T Cells 1
Plasma cells 4
AT2 3
Smooth muscle cells 5
EC General capillary 6
Dendritic cells 3


/tmp/ipykernel_3780808/3929356287.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['cell type']=ct
